# Bike Share Rebalancing With Mathematical Optimization

Bike share systems have become an effective commuting method globally for everyday urban dwellers as well as tourists.

Citi-Bike in NYC being the largest Bike-Sharing network had 1,588 active stations and 25,575 active bikes in July 2022.

Over 3 million rides were completed July 2022 that cover NYC/Hoboken/Jersey City, with around 150,000 active annual members.

During rush hours there are many bike stations that have a high demand for bikes, which means their out-flow of bikes is greater than their in-flow in these stations. 

Meanwhile there are stations that have a high demand for docks (riders return their bikes to these stations) which means their in-flow of bikes is greater than their out-flow.

Lack of available bikes or docks in high-demand stations can cause major imbalance in the bike sharing network and result in customer dissatisfaction and lost revenue.

To tackle this problem, bikes are relocated between stations to create a balance between supply and demand.

## Problem Statement and Solution Approach

Using historical Citi-bike data in NYC and Jersey area during July 2022, we like to know:
- What is the demand for bikes per hour at each station during the first week of August?
- Knowing the demand, how can we minimize loss of sale?

Loss of sale is caused by lack of bikes when customers demand them. So, bikes should be transferred from stations with higher in-flow of bikes to those with higher out-flow of bikes. 

So, first, number of bikes to be added to or removed from each station during each hour should be determined. Then, the physical transfer of bikes between stations should be scheduled. 

In this notebook, we'll focus on the first part and at the end, discuss how the second part can be solved.
We'll use a mixture of Machine Learning (ML) and Mathematical Optimization (MO) to solve this problem. 

**Solution Approach**
The solution approach is comprised of two steps:
- **Step 1**: We use the historical Citi-bike data in NYC and Jersey area during July 2022 and use an ML model to predict the number of in-flow and out-flow of bikes per hour at each station for the first week of August. This is done in [predict_bike_flow](predict_bike_flow.ipynb) Notebook.
- **Step 2**: We use an MO model to decide how many bikes should be added to or removed from each station during each hour so that the total loss of sale is minimized.

To ensure that everyone can run the notebook with the Gurobi restricted license, we reduce the size of the data. To achieve that, we focus on the top 50 stations during the morning rush hours (7 am to 9 am).

The top stations are chosen using the PageRank algorithm.

# Install Required Packages

In [ ]:
%pip install gurobipy
%pip install pandas

# Import Packages

In [ ]:
import datetime
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# Optimization Problem

## Problem Definition

We want to minimize the total loss of sale. Loss of sale at each station and in each hour can be defined as the difference between the total demand of bikes (number of bikes that start their trip from the station) and total supply of bikes.

Total supply is comprised of number of bikes that end their trip at the station plus all the existing bikes at the station (a.k.a inventory) plus number of bikes that are added or removed from that station in that hour through some bike transfers. 

**Assumptions:**
- Inventory at the beginning of first hour (in our case, hour 7) is zero.
- At any given hour, we have access to a limited number of bikes that can be added to the stations in hope of helping reduce the imbalance without yet transferring the bikes between stations (since this analysis is during morning rush hours).

## Load Required Data

In [ ]:
stations = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/bike_share/top_stations.csv', index_col='station')
# run locally
# stations = pd.read_csv('top_stations.csv', index_col='station')
stations.head()

In [ ]:
stations_flow = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/bike_share/stations_flow.csv')
# run locally
# stations_flow = pd.read_csv('stations_flow.csv')
stations_flow['datetime'] = pd.to_datetime(stations_flow['datetime'])
stations_flow.head(25)

The `stations_flow` data contains the prediction for the first 5 days of August 2022 and during all the hours. Our analysis is for morning rush hours, between 7 to 9 am. Also, we can run our MO model daily. For now, we only focus on the first day but at the end, we will show the full model and how it can be run daily.

In [ ]:
# Pandas will give a few SettingWithCopyWarning here when new columns are created. 
# They are false alarms. So, we suppress them.
pd.options.mode.chained_assignment = None
morning_flow = stations_flow[stations_flow['datetime'].dt.hour.between(7, 9)]
morning_flow['date'] = morning_flow['datetime'].dt.date
morning_flow['time'] = morning_flow['datetime'].dt.hour
# For now, let's run the MO model for the first date: 08/01/2022
flow_df = morning_flow.loc[morning_flow['date'] == datetime.date(2022, 8, 1)]
flow_df.set_index(['station', 'time'], inplace=True)
flow_df.head(10)

## Problem Formulation

Let's define our notations for the MO model. We want to run the model for every hour and every station. So, let's define the following two sets:

**Sets**
- $I\quad$: Set of stations
- $T\quad$: Set of hours 

We also have the following information from `stations` and `flow_df` dataframes:

**Parameters**
- $e_{i,t}\quad$: number of bikes that end their trip at station $i$ at hour $t$ (a.k.a. supply)
- $s_{i,t}\quad$: number of bikes that start their trip at station $i$ at hour $t$ (a.k.a. demand)
- $c_{i}\quad$: capacity of station $i$

We know it's not easy to transfer bikes between stations during rush hours and heavy traffic. To mitigate loss due to unavailable bikes at high-demand stations, we assume there is a small reserve of bikes available at the beginning of an hour that we can allocate to the stations. We show that with $N$: 
- $N\quad$: Number of bikes at hand that we can assign to stations at a given hour

In [ ]:
num_bikes = 25  # N: Number of bikes at hand that we can assign to stations at a given hour
station_names = list(stations.index)  # set I
time_rng = morning_flow['time'].drop_duplicates().values  # set T

In [ ]:
station_time = flow_df.index  # pair of (i,t) index
start_forecast = flow_df.start_forecast  # s
end_forecast = flow_df.end_forecast  # e
capacity = stations.capacity  # c

## Decision Variables

First of all, we like to find out how many bikes should be added to or removed from each station during each hour. We show number of bikes added with $y_{i,t}$ and those removed with $z_{i,t}$:
- $y_{i,t}\quad$: number of bikes to be added to station $i$ at hour $t$
- $z_{i,t}\quad$: number of bikes to be removed from station $i$ at hour $t$

The goal of the model is to reduce the lost sale at each station per hour. This value, also depends on the value of decision variables $y_{i,t}$ and $z_{i,t}$ and as a result, is a decision variable itself. We show that with $l_{i,t}$:
- $l_{i,t}\quad$: lost sale at station $i$ at hour $t$

**Any other variables that you can think of?**

With these initial decision variables, we can start the MO model:

In [ ]:
mdl = gp.Model('bike_rebalancing')
# Variables

y = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='y')
# y = mdl.addVars(station_names, time_rng, lb=0, vtype=GRB.CONTINUOUS, name='y')  # alternatively
z = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='z')
l = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='l')

## Constraints

First, we set up lowerbound and upperbound values for the number of bikes that can be added to or removed from a station in each hour.

In each hour, we have $s_{i,t}$ bikes that start their trip from the station and $e_{i,t}$ bikes that end their trip at the station. If $s_{i,t} \ge e_{i,t}$, demand is exceeding supply. In this case, we may choose to add some bikes to this station to reduce the loss (we can also add nothing). One thing to remember is that we cannot add more bikes than the station's capacity.

On the other hand, if $s_{i,t} \le e_{i,t}$, we may choose to remove some of the excess bikes from that station (we can also remove nothing). However, if $e_{i,t} \ge s_{i,t} + c_i$, then we'll have more bikes arriving to the station than even the station capacity. In that case, we must remove some bikes to avoid overflow.

The above descriptions, help us to define the bounds on $y_{i,t}$ and $z_{i,t}$.
For adding bikes, the bounds are:

\begin{align}
&??? &\quad \forall i \in I, t \in T\\
\end{align}

For removing bikes:

\begin{align}
&??? &\quad \forall i \in I, t \in T\\
\end{align}


In [ ]:
# Bounds of Y
... # ???

# Bounds of z
... # ???

Next, we set up the initial inventory (i.e. inventory at hour $t_0$) to be 0. 

\begin{align}
&??? & \\
\end{align}

In [ ]:
t0 = 7
# setting up initial inventory
... # ???

How do you define the inventory at a station at the beginning of an hour? Is there a limit on the amount of inventory?


\begin{align}
&??? &\\
\end{align}



Next, we need to define how the loss of sale is calculated. Loss of sale is the difference between the demand and all the supply of bikes at a station.
- Demand of bikes are all the bikes that leave a station, in any shape or form. So, what are the demands?
- Supply of bikes are all the bikes that arrive at a station, in any shape or form. So, what are the supplies?

Of course, if the supply is greater than the demand, there is no loss. So, we need to ensure that loss only considers non-negative values. This can be achieved by: 

\begin{align}
&??? &\quad \forall i \in I, t \in T \\
\end{align}

In [ ]:
# loss definition
... # ???

We assumed that we have a small reserve of bikes at the beginning of each hour to allocate to stations. This limit is on the total number of bikes added to the stations. 

What do you think we need to do to add this constraint?


\begin{align}
&??? \\
\end{align}

In [ ]:
# limit on number of bikes added
... # ???

## Objective

The objective is to minimize total loss of sale. 

$$\min \sum_{i,t} l_{i,t}$$

In [ ]:
mdl.setObjective(l.sum(), GRB.MINIMIZE)

We can now tell Gurobi that the model is complete and it can solve the problem.

In [ ]:
mdl.optimize()

## Post Processing

In [ ]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in y.items():
        df.loc[k, 'bikes_added'] = v.x
        df.loc[k, 'bikes_removed'] = z[k].x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time')['bikes_added'].sum()
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

# Model Enhancement

Looking at the `output_df`, you may notice stations that see a transfer of bikes (addition or removal) at every hour. Even worse are those stations where some bikes are removed from them in one hour but then bikes are added to them in the next hour. We know these additions/removals consume time and money. What can we do to avoid this situation?

One way to formulate this is to introduce a fixed cost for the use of the truck (or the bicycle trailer) that transfers the bikes and then add this term to the objective function. With a cost associated with the transfer, the model is incentivized to use fewer number of transfers.

So first, we should calculate the number of times a transfer has occurred. Any time bikes are added to a station or removed from a station, a transfer has happened. So, we need a way to link addition of bikes and removal of bikes to a transfer. One way to achieve this is to introduce two new binary variables as follows:

- $x_{i,t}\quad$: 1 if any bike is added to station $i$ at hour $t$; 0 otherwise
- $w_{i,t}\quad$: 1 if any bike is removed from station $i$ at hour $t$; 0 otherwise

Next, we need to establish the relationship between $y_{i,t}$ with $x_{i,t}$, and $z_{i,t}$ with $w_{i,t}$. Basically, we want to say:
if $y_{i,t} \ge 0$, then $x_{i,t} = 1$ and if $z_{i,t} \ge 0$, then $w_{i,t} = 1$. 

We introduce the following two constraints:

\begin{align}
&y_{i,t} \le M x_{i,t} &\quad \forall i \in I, t \in T \tag{11}\\
&z_{i,t} \le M w_{i,t} &\quad \forall i \in I, t \in T \tag{12}\\
\end{align}

where $M$ is a large number.

Constraint 11 ensures that if $y_{i,t}\ge 0$, then $x_{i,t} = 1$. But by itself, this constraint cannot make $x_{i,t} =0$ if $y_{i,t} \le 0$. The same is true with constraint 12. It ensures that $z_{i,t} \ge 0$ make $w_{i,t} = 1$. However, it cannot force $w_{i,t} = 0$ if $z_{i,t} \ge 0$.

This can be achieved by the objective function.

Total number of transfers is equal to sum of $x_{i,t}$ and $w_{i,t}$ and our goal is to minimize number of transfers. So, we add these terms to the objective function. In other words, our new objective function is:

$$\min \sum_{i,t} (l_{i,t} + x_{i,t} + w_{i,t})$$

Since minimizing total transfers is desired, the objective function tries to make both $x_{i,t}$ and $w_{i,t}$ as small as possible (or zero, in this case). Along with constraints 11 and 12, this means that for cases where $x_{i,t}$ and $w_{i,t}$ can take either 0 or 1, objective function forces them to get a value of 0. Moreover, since any extra transfer causes either $x_{i,t}$ or $w_{i,t}$ to be 1, the model is incentivized to have fewer transfers in order to minimize the objective function.

Of course, you can make this model even more generic by having a coefficient for each term in the objective function (think of them as cost).

In [ ]:
x = mdl.addVars(station_time, vtype=GRB.BINARY, name='x')  # 1 if y_{i,t} >= 0 
w = mdl.addVars(station_time, vtype=GRB.BINARY, name='w')  # 1 if z_{i,t} >= 0

big_m = 1000  # large number
# relation between y and x
mdl.addConstrs((y[i, t] <= big_m * x[i, t] for i, t in station_time), 'rel_y_x')
# relation between z and w
mdl.addConstrs((z[i, t] <= big_m * w[i, t] for i, t in station_time), 'rel_z_w')

# new objective
obj = l.sum() + (x.sum() + w.sum())
mdl.setObjective(obj, GRB.MINIMIZE)
mdl.optimize()

## Post Processing

In [ ]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in y.items():
        df.loc[k, 'bikes_added'] = v.x
        df.loc[k, 'bikes_removed'] = z[k].x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time')['bikes_added'].sum()
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

# Extra

## Scenario Analysis

An important requirements in many MO problems is scenario analysis or what-if analysis. 
Generally speaking, in what-if analysis, we're interested in knowing how the solution changes under various scenarios. 
Think about our case here.
- How does the solution change if the number of reserved bikes increase or decrease by 10%?
- In the enhanced model, what happens if the cost of lost sale change? How about the cost of visiting a location for adding or removing the bikes?
- What if a new station is added close to the busiest station?
- What if we want to ensure that every station have at least 2 available bikes at the beginning of each hour?

Our model here is still a simple model. But you can imagine the value that the scenario analysis can provide. It can enable you to answer many questions by creating and comparing different scenarios and evaluating their outcomes, so that you can assess their impacts on the business goals. To learn more, you can check this [multi-scenario example](https://www.gurobi.com/documentation/9.5/examples/multiscenario_py.html) from Gurobi.

## How this problem is solved in reality?

After knowing how many bikes needed in each station, the bikes need to be physically moved from one station to another. 

During rush hours, the traffic is already heavy. So, bicycle trailers (that can usually hold 5 bicycles) are used to move bikes around.

During lighter hours (mainly at night), the bikes are transferred using trucks.

In either case, some bikes should be removed from stations where there are more in-flow of bikes and should be transferred to stations where there are more out-flow of bikes to balance them out. 
This problem, where trucks need to go from one station to another and either pick up bikes or deliver them, is itself another mathematical optimization problem. 

In this problem, we need to ensure that all the stations that have a pickup or delivery, are visited during a certain time window and the goal can be to do this with minimum number of trucks or minimum transportation cost (for example, fuel cost plus the cost for using the truck). This problem is a variation of the famous Vehicle Routing Problem (VRP) with pickup and delivery.
To learn more, check out [this webinar](https://www.gurobi.com/resource/how-to-synchronize-complex-routing-operations-synched-vrps-with-gurobi/)